In [74]:
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error, r2_score
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
df = pd.read_csv('/content/drive/My Drive/Resources/kingcounty/seattle_sales_cleaned_data.csv')
# Review the DataFrame
df

,id,sale_date,sale_price,latitude,longitude,city,zoning,land_val,imp_val,year_built,sqft_lot,sqft,grade,fbsmt_grade,home_condition,stories,beds,bath_full,bath_3qtr,bath_half,garb_sqft,gara_sqft,wfnt,golf,greenbelt,noise_traffic,view_rainier,view_olympics,view_cascades,view_territorial,view_skyline,view_sound,view_lakewash,view_otherwater,year
0,0,1/2/2018,665000,47.559358,-122.317370,SEATTLE,SF 5000,41000,122000,1920,2525,1570,6,4,4,1.5,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2018
1,1,1/9/2018,499000,47.524993,-122.316475,SEATTLE,SF 5000,29000,55000,1907,6000,1350,5,0,3,1.5,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2018
2,2,2/14/2018,675000,47.624203,-122.300762,SEATTLE,SF 5000,97000,132000,1908,5286,1800,7,5,4,1.0,4,3,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,2018
3,3,2/13/2018,1150000,47.721276,-122.335857,SEATTLE,SF 7200,440000,221000,1920,47916,1590,7,0,3,1.5,3,1,0,0,0,0,8,0,0,2,0,0,0,0,0,0,0,4,2018
4,4,2/14/2018,705000,47.666210,-122.324618,SEATTLE,SF 5000,81000,72000,1918,3825,780,6,0,3,1.0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24933,24945,12/8/2022,895000,47.689100,-122.332842,SEATTLE,NR3,440000,327000,1926,3000,1100,7,7,3,1.5,3,2,0,0,370,0,0,0,0,0,0,0,0,0,0,0,0,0,2022
24934,24946,12/5/2022,1900000,47.595587,-122.291790,SEATTLE,NR3,790000,1432000,1924,5016,3000,10,8,3,2.0,4,2,1,0,240,0,0,0,0,0,0,0,2,2,0,0,1,0,2022
24935,24947,12/17/2022,1750000,47.629275,-122.307670,SEATTLE,NR3,853000,727000,1908,4800,2900,9,8,3,1.5,4,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,2022
24936,24948,12/15/2022,1250000,47.676524,-122.392174,SEATTLE,NR3,497000,532000,1928,3825,1730,8,8,3,1.0,3,1,1,0,180,0,0,0,0,0,0,0,0,0,0,0,0,0,2022


In [23]:
df['sale_date'] = pd.to_datetime(df['sale_date'])
df['sale_month'] = df['sale_date'].dt.month
df['sale_day'] = df['sale_date'].dt.day
df.rename(columns={'year': 'sale_year'}, inplace=True)
df.head()

,id,sale_date,sale_price,latitude,longitude,city,zoning,land_val,imp_val,year_built,sqft_lot,sqft,grade,fbsmt_grade,home_condition,stories,beds,bath_full,bath_3qtr,bath_half,garb_sqft,gara_sqft,wfnt,golf,greenbelt,noise_traffic,view_rainier,view_olympics,view_cascades,view_territorial,view_skyline,view_sound,view_lakewash,view_otherwater,sale_year,sale_month,sale_day
0,0,2018-01-02,665000,47.559358,-122.317370,SEATTLE,SF 5000,41000,122000,1920,2525,1570,6,4,4,1.5,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2018,1,2
1,1,2018-01-09,499000,47.524993,-122.316475,SEATTLE,SF 5000,29000,55000,1907,6000,1350,5,0,3,1.5,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2018,1,9
2,2,2018-02-14,675000,47.624203,-122.300762,SEATTLE,SF 5000,97000,132000,1908,5286,1800,7,5,4,1.0,4,3,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,2018,2,14
3,3,2018-02-13,1150000,47.721276,-122.335857,SEATTLE,SF 7200,440000,221000,1920,47916,1590,7,0,3,1.5,3,1,0,0,0,0,8,0,0,2,0,0,0,0,0,0,0,4,2018,2,13
4,4,2018-02-14,705000,47.666210,-122.324618,SEATTLE,SF 5000,81000,72000,1918,3825,780,6,0,3,1.0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2018,2,14


In [24]:
#From the df, select all sales prices above 1,000,000 into one dataframe, and the rest into another one
#Re-run everything again the same way except use the new dfs

In [25]:
less_df = df[df["sale_price"] < 1000000]
less_df

,id,sale_date,sale_price,latitude,longitude,city,zoning,land_val,imp_val,year_built,sqft_lot,sqft,grade,fbsmt_grade,home_condition,stories,beds,bath_full,bath_3qtr,bath_half,garb_sqft,gara_sqft,wfnt,golf,greenbelt,noise_traffic,view_rainier,view_olympics,view_cascades,view_territorial,view_skyline,view_sound,view_lakewash,view_otherwater,sale_year,sale_month,sale_day
0,0,2018-01-02,665000,47.559358,-122.317370,SEATTLE,SF 5000,41000,122000,1920,2525,1570,6,4,4,1.5,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2018,1,2
1,1,2018-01-09,499000,47.524993,-122.316475,SEATTLE,SF 5000,29000,55000,1907,6000,1350,5,0,3,1.5,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2018,1,9
2,2,2018-02-14,675000,47.624203,-122.300762,SEATTLE,SF 5000,97000,132000,1908,5286,1800,7,5,4,1.0,4,3,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,2018,2,14
4,4,2018-02-14,705000,47.666210,-122.324618,SEATTLE,SF 5000,81000,72000,1918,3825,780,6,0,3,1.0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2018,2,14
5,5,2018-02-15,650000,47.672576,-122.284472,SEATTLE,SF 5000,93000,100000,1948,6750,830,7,0,4,1.0,2,1,0,0,230,0,0,0,0,2,0,0,0,0,0,0,0,0,2018,2,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24907,24919,2022-09-30,999950,47.514481,-122.257870,SEATTLE,NR3,176000,686000,1907,5650,2980,8,8,3,1.5,6,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2022,9,30
24920,24932,2022-11-01,780000,47.564364,-122.365854,SEATTLE,NR3,266000,432000,1920,5360,1660,7,7,3,1.0,3,2,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2022,11,1
24926,24938,2022-11-11,675000,47.719362,-122.330710,SEATTLE,NR2,523000,310000,1953,7938,1310,7,0,3,1.0,3,1,0,0,0,260,0,0,0,0,0,0,0,0,0,0,0,0,2022,11,11
24932,24944,2022-12-06,845000,47.712488,-122.324969,SEATTLE,NR2,367000,474000,1945,7320,2330,7,6,3,1.0,3,2,1,0,370,0,0,0,0,1,0,0,0,0,0,0,0,0,2022,12,6


In [26]:
less_df['zoning'].value_counts()
category_mapping = {
    'NR3': 1,
    'NR2': 2,
    'SF 5000': 3,
    'RSL (M)': 4,
    'SF 7200': 5,
    'NR1': 6,
    'SF 9600': 7
}

# Replace categorical values with integer values using the mapping
less_df['zoning'] = less_df['zoning'].map(category_mapping)
less_df['zoning'].value_counts()

<ipython-input-26-29e8cdb7a623>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  less_df['zoning'] = less_df['zoning'].map(category_mapping)


1    10230
2     2810
4     1103
3      740
5      162
6       79
7        6
Name: zoning, dtype: int64

In [27]:
less_df['sale_date'] = pd.to_datetime(df['sale_date'])
less_df['sale_date'] = less_df['sale_date'].astype('int64') // 10**9
less_df['sale_date']

<ipython-input-27-8681f4f75194>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  less_df['sale_date'] = pd.to_datetime(df['sale_date'])
<ipython-input-27-8681f4f75194>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  less_df['sale_date'] = less_df['sale_date'].astype('int64') // 10**9


0        1514851200
1        1515456000
2        1518566400
4        1518566400
5        1518652800
            ...    
24907    1664496000
24920    1667260800
24926    1668124800
24932    1670284800
24933    1670457600
Name: sale_date, Length: 15130, dtype: int64

In [28]:
less_df = less_df.drop(columns=['city'])
less_df

,id,sale_date,sale_price,latitude,longitude,zoning,land_val,imp_val,year_built,sqft_lot,sqft,grade,fbsmt_grade,home_condition,stories,beds,bath_full,bath_3qtr,bath_half,garb_sqft,gara_sqft,wfnt,golf,greenbelt,noise_traffic,view_rainier,view_olympics,view_cascades,view_territorial,view_skyline,view_sound,view_lakewash,view_otherwater,sale_year,sale_month,sale_day
0,0,1514851200,665000,47.559358,-122.317370,3,41000,122000,1920,2525,1570,6,4,4,1.5,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2018,1,2
1,1,1515456000,499000,47.524993,-122.316475,3,29000,55000,1907,6000,1350,5,0,3,1.5,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2018,1,9
2,2,1518566400,675000,47.624203,-122.300762,3,97000,132000,1908,5286,1800,7,5,4,1.0,4,3,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,2018,2,14
4,4,1518566400,705000,47.666210,-122.324618,3,81000,72000,1918,3825,780,6,0,3,1.0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2018,2,14
5,5,1518652800,650000,47.672576,-122.284472,3,93000,100000,1948,6750,830,7,0,4,1.0,2,1,0,0,230,0,0,0,0,2,0,0,0,0,0,0,0,0,2018,2,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24907,24919,1664496000,999950,47.514481,-122.257870,1,176000,686000,1907,5650,2980,8,8,3,1.5,6,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2022,9,30
24920,24932,1667260800,780000,47.564364,-122.365854,1,266000,432000,1920,5360,1660,7,7,3,1.0,3,2,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2022,11,1
24926,24938,1668124800,675000,47.719362,-122.330710,2,523000,310000,1953,7938,1310,7,0,3,1.0,3,1,0,0,0,260,0,0,0,0,0,0,0,0,0,0,0,0,2022,11,11
24932,24944,1670284800,845000,47.712488,-122.324969,2,367000,474000,1945,7320,2330,7,6,3,1.0,3,2,1,0,370,0,0,0,0,1,0,0,0,0,0,0,0,0,2022,12,6


In [29]:
less_df = less_df.drop(columns=['sale_date'])
less_df = less_df.drop(columns=['id'])

In [30]:
less_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15130 entries, 0 to 24933
Data columns (total 34 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   sale_price        15130 non-null  int64  
 1   latitude          15130 non-null  float64
 2   longitude         15130 non-null  float64
 3   zoning            15130 non-null  int64  
 4   land_val          15130 non-null  int64  
 5   imp_val           15130 non-null  int64  
 6   year_built        15130 non-null  int64  
 7   sqft_lot          15130 non-null  int64  
 8   sqft              15130 non-null  int64  
 9   grade             15130 non-null  int64  
 10  fbsmt_grade       15130 non-null  int64  
 11  home_condition    15130 non-null  int64  
 12  stories           15130 non-null  float64
 13  beds              15130 non-null  int64  
 14  bath_full         15130 non-null  int64  
 15  bath_3qtr         15130 non-null  int64  
 16  bath_half         15130 non-null  int64 

In [31]:
# Specify the column to exclude from the subset
column_to_exclude = 'sale_day'

# Create a subset of columns by excluding the specified column
subset_columns = [col for col in less_df.columns if col != column_to_exclude]

# Create a new DataFrame with the subset of columns
df_subset = less_df[subset_columns]

# Remove duplicate rows based on the subset of columns
df_no_duplicates = df_subset.drop_duplicates()
less_df = df_no_duplicates

In [13]:
summary_stats = less_df.describe()
print(summary_stats)

                 id     sale_price      latitude     longitude        zoning  \
count  15130.000000   15130.000000  15130.000000  15130.000000  15130.000000   
mean   11822.506081  722548.167548     47.614869   -122.336379      1.573562   
std     6550.313533  163015.381120      0.076006      0.042307      1.012273   
min        0.000000   65000.000000     47.495701   -122.416410      1.000000   
25%     6298.250000  610000.000000     47.542605   -122.374107      1.000000   
50%    11260.000000  735000.000000     47.605695   -122.347071      1.000000   
75%    17166.750000  850000.000000     47.687757   -122.298319      2.000000   
max    24945.000000  999999.000000     47.733910   -122.238838      7.000000   

           land_val       imp_val    year_built       sqft_lot          sqft  \
count  1.513000e+04  1.513000e+04  15130.000000   15130.000000  15130.000000   
mean   3.905037e+05  3.736561e+05   1942.553602    5784.326570   1566.112360   
std    1.716542e+05  1.746714e+05     2

In [88]:
columns = ['sale_price', 'sqft_lot', 'sqft', 'land_val', 'imp_val']

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# Calculate IQR for the specified columns
Q1 = less_df[columns].quantile(0.25)
Q3 = less_df[columns].quantile(0.75)
IQR = Q3 - Q1

# Define lower and upper bounds for the specified columns
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter outliers for the specified columns
filtered_df = less_df[~((less_df[columns] < lower_bound) | (less_df[columns] > upper_bound)).any(axis=1)]
filtered_df = filtered_df.reset_index(drop=True)
filtered_df.head()

,sale_price,latitude,longitude,zoning,land_val,imp_val,year_built,sqft_lot,sqft,grade,fbsmt_grade,home_condition,stories,beds,bath_full,bath_3qtr,bath_half,garb_sqft,gara_sqft,wfnt,golf,greenbelt,noise_traffic,view_rainier,view_olympics,view_cascades,view_territorial,view_skyline,view_sound,view_lakewash,view_otherwater,sale_year,sale_month
0,665000,47.56,-122.32,3,41000,122000,1920,2525,1570,6,4,4,1.50,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2018,1
1,499000,47.52,-122.32,3,29000,55000,1907,6000,1350,5,0,3,1.50,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2018,1
2,675000,47.62,-122.30,3,97000,132000,1908,5286,1800,7,5,4,1.00,4,3,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,2018,2
3,705000,47.67,-122.32,3,81000,72000,1918,3825,780,6,0,3,1.00,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2018,2
4,650000,47.67,-122.28,3,93000,100000,1948,6750,830,7,0,4,1.00,2,1,0,0,230,0,0,0,0,2,0,0,0,0,0,0,0,0,2018,2


In [130]:
summary_stats = filtered_df.describe()
print(summary_stats)

       sale_price  latitude  longitude   zoning  land_val   imp_val  \
count    14291.00  14291.00   14291.00 14291.00  14291.00  14291.00   
mean    721213.02     47.62    -122.34     1.56 383951.64 369080.90   
std     159786.32      0.08       0.04     0.99 161125.84 157031.14   
min     250000.00     47.50    -122.42     1.00      0.00      0.00   
25%     608000.00     47.54    -122.37     1.00 274000.00 264000.00   
50%     732000.00     47.61    -122.35     1.00 372000.00 369000.00   
75%     850000.00     47.69    -122.30     2.00 492000.00 476000.00   
max     999999.00     47.73    -122.24     7.00 826000.00 805000.00   

       year_built  sqft_lot     sqft    grade  fbsmt_grade  home_condition  \
count    14291.00  14291.00 14291.00 14291.00     14291.00        14291.00   
mean      1942.04   5543.93  1534.27     6.87         3.37            3.69   
std         24.55   1784.21   522.96     0.67         3.23            0.78   
min       1900.00    773.00   260.00     3.00   

In [89]:
y = filtered_df["sale_price"].ravel()
scaler = StandardScaler()
y_scaled = scaler.fit_transform(y.reshape(-1, 1)).ravel()
X = filtered_df.copy()
X.drop("sale_price", axis=1, inplace=True)

In [90]:
y_scaled[:5]

array([-0.35181351, -1.39073732, -0.28922774, -0.10147042, -0.44569217])

In [91]:
X.dropna()
X.head()

,latitude,longitude,zoning,land_val,imp_val,year_built,sqft_lot,sqft,grade,fbsmt_grade,home_condition,stories,beds,bath_full,bath_3qtr,bath_half,garb_sqft,gara_sqft,wfnt,golf,greenbelt,noise_traffic,view_rainier,view_olympics,view_cascades,view_territorial,view_skyline,view_sound,view_lakewash,view_otherwater,sale_year,sale_month
0,47.56,-122.32,3,41000,122000,1920,2525,1570,6,4,4,1.50,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2018,1
1,47.52,-122.32,3,29000,55000,1907,6000,1350,5,0,3,1.50,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2018,1
2,47.62,-122.30,3,97000,132000,1908,5286,1800,7,5,4,1.00,4,3,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,2018,2
3,47.67,-122.32,3,81000,72000,1918,3825,780,6,0,3,1.00,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2018,2
4,47.67,-122.28,3,93000,100000,1948,6750,830,7,0,4,1.00,2,1,0,0,230,0,0,0,0,2,0,0,0,0,0,0,0,0,2018,2


In [98]:
X_train, X_test, y_train, y_test = train_test_split(X, y_scaled, test_size=0.2, random_state=42)
scaler4 = StandardScaler()
X_scaler = scaler4.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [99]:
regression_model = LinearRegression()

# Fit the model using training data
regression_model.fit(X_train_scaled, y_train)

# Predict values using the trained model
y_pred = regression_model.predict(X_test_scaled)

In [100]:
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("Mean Squared Error:", mse)
print("R-squared:", r2)

Mean Squared Error: 0.45111933737927007
R-squared: 0.5489248755571852


In [101]:
rf_model = RandomForestRegressor(n_estimators=200, random_state=42)
rf_model.fit(X_train_scaled, y_train)

RandomForestRegressor(n_estimators=200, random_state=42)

In [102]:
y_pred2 = rf_model.predict(X_test_scaled)

In [103]:
pd.options.display.float_format = '{:.2f}'.format
y_pred2_original = scaler.inverse_transform(y_pred2.reshape(-1, 1)).ravel()

# Inverse transform the scaled actual prices to get original prices
y_test_original = scaler.inverse_transform(y_test.reshape(-1, 1)).ravel()

mse = mean_squared_error(y_test_original, y_pred2_original)
rmse = np.sqrt(mse)
r2 = r2_score(y_test_original, y_pred2_original)
rmse_rounded = round(rmse) # Round and convert to integer

absolute_diff = np.abs(y_test_original - y_pred2_original)

# Create a DataFrame for comparison
comparison_df = pd.DataFrame({
    'Actual Price': y_test_original,
    'Predicted Price': y_pred2_original,
    'Absolute Difference': absolute_diff
})

# Print the comparison DataFrame
print(comparison_df)
print("Root Mean Squared Error:", rmse_rounded)
print("R-squared:", r2)

      Actual Price  Predicted Price  Absolute Difference
0        840000.00        877461.04             37461.04
1        565000.00        542525.38             22474.62
2        707000.00        710612.93              3612.93
3        825000.00        867352.74             42352.74
4        516000.00        566113.95             50113.95
...            ...              ...                  ...
2854     710000.00        826029.44            116029.44
2855     734900.00        809526.85             74626.85
2856     800000.00        766593.85             33406.15
2857     860000.00        868192.05              8192.05
2858     606321.00        620271.66             13950.66

[2859 rows x 3 columns]
Root Mean Squared Error: 94367
R-squared: 0.651224505673383


In [45]:
filtered_df2 = filtered_df[['sale_price', 'latitude' , 'longitude',	'zoning','land_val','imp_val','year_built','sqft_lot','sqft','grade','fbsmt_grade','home_condition','stories','beds','bath_full', 'sale_year', 'sale_month']]
filtered_df2.head()

,sale_price,latitude,longitude,zoning,land_val,imp_val,year_built,sqft_lot,sqft,grade,fbsmt_grade,home_condition,stories,beds,bath_full,sale_year,sale_month
0,665000,47.56,-122.32,3,41000,122000,1920,2525,1570,6,4,4,1.50,4,1,2018,1
1,499000,47.52,-122.32,3,29000,55000,1907,6000,1350,5,0,3,1.50,4,1,2018,1
2,675000,47.62,-122.30,3,97000,132000,1908,5286,1800,7,5,4,1.00,4,3,2018,2
3,705000,47.67,-122.32,3,81000,72000,1918,3825,780,6,0,3,1.00,1,1,2018,2
4,650000,47.67,-122.28,3,93000,100000,1948,6750,830,7,0,4,1.00,2,1,2018,2


In [58]:
y = filtered_df2["sale_price"].ravel()
scaler = StandardScaler()
y_scaled = scaler.fit_transform(y.reshape(-1, 1)).ravel()
X = filtered_df2.copy()
X.drop("sale_price", axis=1, inplace=True)
X_train, X_test, y_train, y_test = train_test_split(X, y_scaled, test_size=0.2, random_state=42)
scaler4 = StandardScaler()
X_scaler = scaler4.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
rf_model = RandomForestRegressor(n_estimators=100, max_depth = None, random_state=42)
rf_model.fit(X_train_scaled, y_train)
y_pred2 = rf_model.predict(X_test_scaled)

In [86]:
min_samples_split_value = 10
min_samples_leaf_value = 5

# Create and train a RandomForestRegressor with the chosen hyperparameters
rf_model = RandomForestRegressor(n_estimators=200, max_depth=None,
                                 min_samples_split=min_samples_split_value,
                                 min_samples_leaf=min_samples_leaf_value,
                                 random_state=42)

rf_model.fit(X_train_scaled, y_train)
y_pred2 = rf_model.predict(X_test_scaled)

In [87]:
pd.options.display.float_format = '{:.2f}'.format
y_pred2_original = scaler.inverse_transform(y_pred2.reshape(-1, 1)).ravel()

# Inverse transform the scaled actual prices to get original prices
y_test_original = scaler.inverse_transform(y_test.reshape(-1, 1)).ravel()

mse = mean_squared_error(y_test_original, y_pred2_original)
rmse = np.sqrt(mse)
rmse = round(rmse, 2)
r2 = r2_score(y_test_original, y_pred2_original) # Round and convert to integer

absolute_diff = np.abs(y_test_original - y_pred2_original)

#Create a DataFrame for comparison
comparison_df = pd.DataFrame({
    'Actual Price': y_test_original,
    'Predicted Price': y_pred2_original,
    'Absolute Difference': absolute_diff
})

# Print the comparison DataFrame
print(comparison_df)
print("Root Mean Squared Error:", rmse)
print("R-squared:", r2)

      Actual Price  Predicted Price  Absolute Difference
0        840000.00        872274.85             32274.85
1        565000.00        528786.98             36213.02
2        707000.00        677444.43             29555.57
3        825000.00        893007.68             68007.68
4        516000.00        566487.94             50487.94
...            ...              ...                  ...
2854     710000.00        827816.94            117816.94
2855     734900.00        829963.93             95063.93
2856     800000.00        721293.58             78706.42
2857     860000.00        861075.15              1075.15
2858     606321.00        612791.37              6470.37

[2859 rows x 3 columns]
Root Mean Squared Error: 94913.5
R-squared: 0.6471707078412317


In [129]:
from sklearn.ensemble import GradientBoostingRegressor
y = filtered_df2["sale_price"].ravel()
scaler2 = StandardScaler()
y_scaled = scaler2.fit_transform(y.reshape(-1, 1)).ravel()
X = filtered_df2.drop("sale_price", axis=1)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y_scaled, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create and train a GradientBoostingRegressor
gb_model = GradientBoostingRegressor(n_estimators=200, learning_rate=0.1, min_samples_split = 10, min_samples_leaf = 5, random_state=42)
gb_model.fit(X_train_scaled, y_train)

# Make predictions on the test set
y_pred2 = gb_model.predict(X_test_scaled)

pd.options.display.float_format = '{:.2f}'.format
y_pred2_original = scaler2.inverse_transform(y_pred2.reshape(-1, 1)).ravel()

# Inverse transform the scaled actual prices to get original prices
y_test_original = scaler2.inverse_transform(y_test.reshape(-1, 1)).ravel()

mse = mean_squared_error(y_test_original, y_pred2_original)
rmse = np.sqrt(mse)
rmse = round(rmse, 2)
r2 = r2_score(y_test_original, y_pred2_original) # Round and convert to integer

absolute_diff = np.abs(y_test_original - y_pred2_original)

#Create a DataFrame for comparison
comparison_df = pd.DataFrame({
    'Actual Price': y_test_original,
    'Predicted Price': y_pred2_original,
    'Absolute Difference': absolute_diff
})

# Print the comparison DataFrame
print(comparison_df)
print("Root Mean Squared Error:", rmse)
print("R-squared:", r2)

      Actual Price  Predicted Price  Absolute Difference
0        840000.00        908496.93             68496.93
1        565000.00        565777.58               777.58
2        707000.00        722389.52             15389.52
3        825000.00        855324.69             30324.69
4        516000.00        574199.22             58199.22
...            ...              ...                  ...
2854     710000.00        814922.87            104922.87
2855     734900.00        824865.85             89965.85
2856     800000.00        791001.33              8998.67
2857     860000.00        903660.95             43660.95
2858     606321.00        642636.07             36315.07

[2859 rows x 3 columns]
Root Mean Squared Error: 90250.4
R-squared: 0.68098806425483


In [119]:
import xgboost as xgb
y = filtered_df2["sale_price"].ravel()
scaler = StandardScaler()
y_scaled = scaler.fit_transform(y.reshape(-1, 1)).ravel()
X = filtered_df2.drop("sale_price", axis=1)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y_scaled, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create and train an XGBoost model
xgb_model = xgb.XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
xgb_model.fit(X_train_scaled, y_train)

# Make predictions on the test set
y_pred = xgb_model.predict(X_test_scaled)

# Calculate evaluation metrics
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("Root Mean Squared Error:", rmse)
print("R-squared:", r2)

Root Mean Squared Error: 0.5657560791729802
R-squared: 0.6799514290128914


In [75]:
lasso_model = Lasso(alpha=0.1)  # You can adjust the alpha parameter for the strength of regularization
lasso_model.fit(X_train_scaled, y_train)

# Make predictions
y_pred_lasso = lasso_model.predict(X_test_scaled)

In [76]:
mse = mean_squared_error(y_test, y_pred_lasso)
r2 = r2_score(y_test, y_pred_lasso)
print("Mean Squared Error:", mse)
print("R-squared:", r2)

Mean Squared Error: 0.6074261697812314
R-squared: 0.39263336234789825


In [67]:
filtered_df3 = filtered_df[['sale_price', 'sale_year', 'sale_month', 'sqft_lot', 'sqft', 'zoning', 'land_val', 'imp_val']]
filtered_df3.head()

,sale_price,sale_year,sale_month,sqft_lot,sqft,zoning,land_val,imp_val
0,665000,2018,1,2525,1570,3,41000,122000
1,499000,2018,1,6000,1350,3,29000,55000
2,675000,2018,2,5286,1800,3,97000,132000
3,705000,2018,2,3825,780,3,81000,72000
4,650000,2018,2,6750,830,3,93000,100000


In [69]:
y = filtered_df3["sale_price"].ravel()
scaler = StandardScaler()
y_scaled = scaler.fit_transform(y.reshape(-1, 1)).ravel()
X = filtered_df3.copy()
X.drop("sale_price", axis=1, inplace=True)
X_train, X_test, y_train, y_test = train_test_split(X, y_scaled, test_size=0.2, random_state=42)
scaler4 = StandardScaler()
X_scaler = scaler4.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
rf_model = RandomForestRegressor(n_estimators=100, max_depth = None, random_state=42)
rf_model.fit(X_train_scaled, y_train)
y_pred2 = rf_model.predict(X_test_scaled)

In [70]:
pd.options.display.float_format = '{:.2f}'.format
y_pred2_original = scaler.inverse_transform(y_pred2.reshape(-1, 1)).ravel()

# Inverse transform the scaled actual prices to get original prices
y_test_original = scaler.inverse_transform(y_test.reshape(-1, 1)).ravel()

mse = mean_squared_error(y_test_original, y_pred2_original)
rmse = np.sqrt(mse)
rmse = round(rmse, 2)
r2 = r2_score(y_test_original, y_pred2_original) # Round and convert to integer

absolute_diff = np.abs(y_test_original - y_pred2_original)

#Create a DataFrame for comparison
comparison_df = pd.DataFrame({
    'Actual Price': y_test_original,
    'Predicted Price': y_pred2_original,
    'Absolute Difference': absolute_diff
})

# Print the comparison DataFrame
print(comparison_df)
print("Root Mean Squared Error:", rmse)
print("R-squared:", r2)

      Actual Price  Predicted Price  Absolute Difference
0        840000.00        855538.10             15538.10
1        565000.00        541553.00             23447.00
2        707000.00        665381.50             41618.50
3        825000.00        904636.75             79636.75
4        516000.00        579886.50             63886.50
...            ...              ...                  ...
2854     710000.00        825665.00            115665.00
2855     734900.00        805654.50             70754.50
2856     800000.00        706384.00             93616.00
2857     860000.00        866166.90              6166.90
2858     606321.00        615215.00              8894.00

[2859 rows x 3 columns]
Root Mean Squared Error: 97137.67
R-squared: 0.6304407982298458
